In [1]:
from mongo_spark_client import Client
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.types import *

# test ur code in cluster with: master=spark://master:7077
# see print statements in UDFs with: master=local[*]
events_db = Client(collection='event', master='local[*]',
                   uri='mongodb://mongo:27017')

In [2]:
# Read into DF
events_df = events_db.read()\
.select('_id', 'name', 'of_interest', 'location', 
        'keywords', 'hashtags', 'image_urls', 
        F.explode('cluster_ids').alias('cluster_id'))\
.where('start_time_ms >= 1491109633207')

Py4JJavaError: An error occurred while calling o32.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting for a server that matches ReadPreferenceServerSelector{readPreference=primary}. Client view of cluster state is {type=UNKNOWN, servers=[{address=mongo:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.SocketTimeoutException: connect timed out}}]
	at com.mongodb.connection.BaseCluster.createTimeoutException(BaseCluster.java:369)
	at com.mongodb.connection.BaseCluster.selectServer(BaseCluster.java:101)
	at com.mongodb.binding.ClusterBinding$ClusterBindingConnectionSource.<init>(ClusterBinding.java:75)
	at com.mongodb.binding.ClusterBinding$ClusterBindingConnectionSource.<init>(ClusterBinding.java:71)
	at com.mongodb.binding.ClusterBinding.getReadConnectionSource(ClusterBinding.java:63)
	at com.mongodb.operation.CommandOperationHelper.executeWrappedCommandProtocol(CommandOpe

In [ ]:
# Write to db
# NOTE: updates wipe out matched docs so validate fields and types!!
events_db.write(events_df)

In [ ]:
# Helpers

## pretty-print df
def pp(df, limit=10, raw=False):
    print(df.schema)
    print('count: ', df.count())
    if raw:
        return df.show()
    else:
        return df.limit(limit).toPandas().head(limit)